# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,jamestown,42.0970,-79.2353,15.01,91,100,3.60,US,1712770682
1,1,blackmans bay,-43.0167,147.3167,10.29,87,78,3.88,AU,1712770971
2,2,puerto natales,-51.7236,-72.4875,10.25,53,0,7.72,CL,1712770944
3,3,port douglas,-16.4833,145.4667,24.86,85,100,3.24,AU,1712770971
4,4,utrik,11.2278,169.8474,26.68,80,99,10.40,MH,1712770866
...,...,...,...,...,...,...,...,...,...,...
585,585,lerwick,60.1545,-1.1494,7.00,92,100,7.99,GB,1712771062
586,586,bodo,67.2800,14.4050,6.02,97,75,9.26,NO,1712770788
587,587,yanykurgan,43.9150,67.2481,16.09,46,7,3.14,KZ,1712771063
588,588,sampit,-2.5333,112.9500,24.21,99,100,0.49,ID,1712771063


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#%%capture --no-display 
# Configure the map plot
vacation_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    logz = True,
    title = "Vacation Cities",
    color = "City"  
)
# Display the map
vacation_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_nulls = city_data_df[(city_data_df["Max Temp"] >= 18) & (city_data_df["Max Temp"] <= 28)]

#check the counts
ideal_weather_nulls.count()

City_ID       195
City          195
Lat           195
Lng           195
Max Temp      195
Humidity      195
Cloudiness    195
Wind Speed    195
Country       192
Date          195
dtype: int64

In [5]:
# Drop any rows with null values
ideal_weather = ideal_weather_nulls.dropna()

#count the remaining cities
ideal_weather.count()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,port douglas,-16.4833,145.4667,24.86,85,100,3.24,AU,1712770971
4,4,utrik,11.2278,169.8474,26.68,80,99,10.40,MH,1712770866
8,8,heroica caborca,30.7167,-112.1500,25.62,14,0,0.72,MX,1712770680
13,13,menongue,-14.6585,17.6910,19.37,92,97,2.43,AO,1712770973
15,15,kashmar,35.2383,58.4656,19.34,29,22,5.11,IR,1712770974
...,...,...,...,...,...,...,...,...,...,...
576,576,beykonak,36.3257,30.3030,18.06,78,6,2.28,TR,1712771061
578,578,basco,20.4487,121.9702,25.88,78,87,7.44,PH,1712771061
580,580,puerto baquerizo moreno,-0.9000,-89.6000,27.55,83,40,2.06,EC,1712770808
588,588,sampit,-2.5333,112.9500,24.21,99,100,0.49,ID,1712771063


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[['City', 'Country', 'Lat', 'Lng','Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
3,port douglas,AU,-16.4833,145.4667,85,
4,utrik,MH,11.2278,169.8474,80,
8,heroica caborca,MX,30.7167,-112.1500,14,
13,menongue,AO,-14.6585,17.6910,92,
15,kashmar,IR,35.2383,58.4656,29,
...,...,...,...,...,...,...
576,beykonak,TR,36.3257,30.3030,78,
578,basco,PH,20.4487,121.9702,78,
580,puerto baquerizo moreno,EC,-0.9000,-89.6000,83,
588,sampit,ID,-2.5333,112.9500,99,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000

#parameters
categories = "accommodation.hotel"
limit = 20
apiKey = geoapify_key

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params = {
    "categories":categories,
    "limit":limit,
    "filters":f"circle:{lng},{lat},{radius}",
    "bias":f"proximity:{lng},{lat}",
    "apiKey":geoapify_key  
}
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

Starting hotel search
port douglas - nearest hotel: Mantra Aqueous Port Douglas
utrik - nearest hotel: No hotel found
heroica caborca - nearest hotel: Hotel Posada del Desierto
menongue - nearest hotel: No hotel found
kashmar - nearest hotel: خانه معلم کاشمر
hawaiian paradise park - nearest hotel: Lava Tree Tropic Inn
adamstown - nearest hotel: Chez Bianca & Benoit
lihue - nearest hotel: Kauai Palms
farafangana - nearest hotel: Le Concombre Masqué
el amria - nearest hotel: Hotel du Centre
bethel - nearest hotel: Hampton Inn Danbury
edinburgh of the seven seas - nearest hotel: No hotel found
nishinoomote - nearest hotel: ゼウスハウス
kapa'a - nearest hotel: Pono Kai Resort
west island - nearest hotel: Cocos Village Bungalows
holualoa - nearest hotel: Kona Hotel
atchison - nearest hotel: Holiday Inn Express
weno - nearest hotel: High Tide Hotel
flying fish cove - nearest hotel: Christmas Island Lodge
eirunepe - nearest hotel: Hotel JOAFRAN
redwood falls - nearest hotel: OYO Hotel Redwood Falls

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
3,port douglas,AU,-16.4833,145.4667,85,Mantra Aqueous Port Douglas
4,utrik,MH,11.2278,169.8474,80,No hotel found
8,heroica caborca,MX,30.7167,-112.1500,14,Hotel Posada del Desierto
13,menongue,AO,-14.6585,17.6910,92,No hotel found
15,kashmar,IR,35.2383,58.4656,29,خانه معلم کاشمر
...,...,...,...,...,...,...
576,beykonak,TR,36.3257,30.3030,78,Uyguluma Oteli
578,basco,PH,20.4487,121.9702,78,Tawsen Hotel
580,puerto baquerizo moreno,EC,-0.9000,-89.6000,83,Dory's House
588,sampit,ID,-2.5333,112.9500,99,Hotel Sultan


In [12]:
# Configure the map plot
vacation_map_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    logz = True,
    title = "Cities and hotels",
    color = "City",  
    hover_cols = ['Country', 'Hotel Name']
)
# Display the map
vacation_map_hotel


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)

<bound method DataFrame.info of                         City Country      Lat       Lng  Humidity  \
3               port douglas      AU -16.4833  145.4667        85   
4                      utrik      MH  11.2278  169.8474        80   
8            heroica caborca      MX  30.7167 -112.1500        14   
13                  menongue      AO -14.6585   17.6910        92   
15                   kashmar      IR  35.2383   58.4656        29   
..                       ...     ...      ...       ...       ...   
576                 beykonak      TR  36.3257   30.3030        78   
578                    basco      PH  20.4487  121.9702        78   
580  puerto baquerizo moreno      EC  -0.9000  -89.6000        83   
588                   sampit      ID  -2.5333  112.9500        99   
589                   kahama      TZ  -3.8333   32.6000        84   

                      Hotel Name  
3    Mantra Aqueous Port Douglas  
4                 No hotel found  
8      Hotel Posada del Desierto  